# 🌱 Plant Disease Detection: Active Domain Adaptation

## Interactive Experiment Runner

This notebook provides a **user-friendly interface** to run all experiments without using the command line.

---

### 📋 Experiment Overview

| # | Experiment | Purpose | Expected Time |
|---|------------|---------|---------------|
| 1 | Baseline Gap | Establish the Lab→Field accuracy drop | ~5 min |
| 2 | Passive Augmentation | Test strong data augmentation | ~5 min |
| 3 | CutMix | Test CutMix regularization | ~10 min |
| 4 | Active Learning | Compare Random vs Entropy | ~15 min |
| 5 | Hybrid Warm-Start | **Our proposed method** | ~20 min |

---

### 🚀 How to Use

1. **Run cells in order** (Shift+Enter or click ▶️)
2. **Modify parameters** in the configuration cells as needed
3. **View results** displayed after each experiment

---

## 1️⃣ Setup & Configuration

Run this cell first to set up the environment.

In [ ]:
# ============================================================
# SETUP - Run this cell first!
# ============================================================

import sys
import os
from pathlib import Path

# Find project root
notebook_dir = Path(os.getcwd())
if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

# Add experiments to path
experiments_dir = project_root / 'experiments'
sys.path.insert(0, str(experiments_dir))

# Verify setup
print("✅ Setup Complete!")
print(f"📁 Project Root: {project_root}")
print(f"📁 Experiments: {experiments_dir}")

# Check for GPU
import torch
if torch.cuda.is_available():
    print(f"🎮 GPU Available: {torch.cuda.get_device_name(0)}")
else:
    print("💻 Running on CPU (slower but works)")

In [ ]:
# ============================================================
# CONFIGURATION - Modify these settings as needed
# ============================================================

# Dataset settings
DATASET_ROOT = str(project_root.parent / 'dataset')  # Where pv and PlantDoc are located
LAB_FOLDER = 'pv'      # Lab/controlled data
FIELD_FOLDER = 'PlantDoc'        # Field/real-world data
CLASS_NAME = 'Tomato'            # Filter to specific crop (or None for all)

# Training settings
BATCH_SIZE = 16                  # Reduce to 8 if you get memory errors
EPOCHS = 5                       # Increase for better results (but slower)
LEARNING_RATE = 0.001

# Active learning settings
BUDGET_PER_ROUND = 50            # Samples to label each round
NUM_ROUNDS = 4                   # Number of active learning rounds

print("✅ Configuration loaded!")
print(f"📊 Dataset: {DATASET_ROOT}")
print(f"🌿 Class filter: {CLASS_NAME or 'All classes'}")
print(f"⚙️ Batch size: {BATCH_SIZE}, Epochs: {EPOCHS}")

---

## 2️⃣ Experiment 01: Baseline Generalization Gap

**Goal**: Train a model on Lab data (PlantVillage) and measure how much accuracy drops on Field data (PlantDoc).

**Expected Result**: ~60-70% accuracy drop (this is the problem we're solving!)

In [ ]:
# ============================================================
# EXPERIMENT 01: Baseline Gap
# ============================================================

print("="*60)
print("🧪 EXPERIMENT 01: Baseline Generalization Gap")
print("="*60)

from common import (
    TrainingConfig, get_transforms, find_dataset_path,
    FilteredImageFolder, create_data_loaders, create_model,
    Trainer, evaluate_accuracy, get_device, set_seed,
    save_model, MODELS_DIR
)
from torch.utils.data import random_split

# Setup
set_seed(42)
device = get_device()

config = TrainingConfig(
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE
)

# Load data
print("\n📂 Loading datasets...")
transforms_dict = get_transforms(config)
class_filter = [CLASS_NAME] if CLASS_NAME else None

# Lab data
lab_path, lab_val_path = find_dataset_path(Path(DATASET_ROOT), LAB_FOLDER)
if lab_val_path:
    train_dataset = FilteredImageFolder(str(lab_path), transforms_dict['train'], class_filter)
    val_dataset = FilteredImageFolder(str(lab_val_path), transforms_dict['val'], class_filter)
else:
    full_dataset = FilteredImageFolder(str(lab_path), transforms_dict['train'], class_filter)
    train_size = int(0.8 * len(full_dataset))
    train_dataset, val_dataset = random_split(full_dataset, [train_size, len(full_dataset) - train_size])

class_names = train_dataset.classes if hasattr(train_dataset, 'classes') else train_dataset.dataset.classes
num_classes = len(class_names)

# Field data
field_path, _ = find_dataset_path(Path(DATASET_ROOT), FIELD_FOLDER)
field_dataset = FilteredImageFolder(str(field_path), transforms_dict['val'], class_filter)

print(f"✅ Lab train: {len(train_dataset)} images")
print(f"✅ Lab val: {len(val_dataset)} images")
print(f"✅ Field test: {len(field_dataset)} images")
print(f"📋 Classes: {class_names}")

# Create loaders
loaders = create_data_loaders(train_dataset, val_dataset, field_dataset, config)

# Create and train model
print("\n🔧 Creating model...")
model = create_model(num_classes)
model = model.to(device)

print("\n🏋️ Training...")
trainer = Trainer(model, device, config)
model = trainer.train(loaders['train'], loaders['val'], epochs=config.epochs)

# Evaluate
print("\n📊 Evaluating...")
lab_acc = evaluate_accuracy(model, loaders['val'], device, desc="Lab")
field_acc = evaluate_accuracy(model, loaders['test'], device, desc="Field")
gap = lab_acc - field_acc

# Save model
save_model(model, MODELS_DIR / 'baseline_model.pth')

# Results
print("\n" + "="*60)
print("📊 RESULTS: Baseline Gap")
print("="*60)
print(f"🏠 Lab Accuracy:   {lab_acc:.2f}%")
print(f"🌾 Field Accuracy: {field_acc:.2f}%")
print(f"📉 GAP: {gap:.2f}%")
print("="*60)

# Store for comparison
exp01_results = {'lab': lab_acc, 'field': field_acc, 'gap': gap}

---

## 3️⃣ Experiment 02: Passive Augmentation

**Goal**: Test if strong data augmentation during training can improve field robustness.

**Augmentations used**:
- 🔄 Flips, rotations
- ☀️ Brightness, contrast changes
- 🌫️ Blur, noise
- ⬛ Random dropout (occlusion simulation)

In [ ]:
# ============================================================
# EXPERIMENT 02: Passive Augmentation
# ============================================================

print("="*60)
print("🧪 EXPERIMENT 02: Passive Augmentation")
print("="*60)

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import copy

# Strong augmentation pipeline
aug_train = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.GaussNoise(p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    A.CoarseDropout(max_holes=8, max_height=20, max_width=20, p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

aug_val = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Custom dataset with Albumentations
class AugDataset(datasets.ImageFolder):
    def __init__(self, root, aug, class_filter=None):
        super().__init__(root, transform=None)
        self.aug = aug
        if class_filter:
            orig = self.classes.copy()
            matched = [c for c in orig if any(f.lower() in c.lower() for f in class_filter)]
            if matched:
                self.classes = matched
                self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
                self.samples = [(p, self.class_to_idx[orig[i]]) for p, i in self.samples if orig[i] in self.classes]
                self.targets = [s[1] for s in self.samples]
    
    def __getitem__(self, idx):
        path, target = self.samples[idx]
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if img is not None else np.zeros((224,224,3), dtype=np.uint8)
        img = self.aug(image=img)['image']
        return img, target

# Load data with augmentation
print("\n📂 Loading data with strong augmentation...")
filter_cls = [CLASS_NAME] if CLASS_NAME else None

train_ds = AugDataset(str(lab_path), aug_train, filter_cls)
val_ds = AugDataset(str(lab_val_path) if lab_val_path else str(lab_path), aug_val, filter_cls)
field_ds = AugDataset(str(field_path), aug_val, filter_cls)

if not lab_val_path:
    train_size = int(0.8 * len(train_ds))
    train_ds, val_ds = random_split(train_ds, [train_size, len(train_ds) - train_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
field_loader = DataLoader(field_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"✅ Train: {len(train_ds)} | Val: {len(val_ds)} | Field: {len(field_ds)}")

# Train
print("\n🏋️ Training with augmentation...")
model2 = create_model(num_classes).to(device)
optimizer = optim.Adam(model2.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

best_acc = 0
best_weights = None

for epoch in range(EPOCHS):
    model2.train()
    correct = total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    
    train_acc = correct / total
    val_acc = evaluate_accuracy(model2, val_loader, device) / 100
    print(f"Epoch {epoch+1}/{EPOCHS} | Train: {train_acc:.4f} | Val: {val_acc:.4f}")
    
    if val_acc > best_acc:
        best_acc = val_acc
        best_weights = copy.deepcopy(model2.state_dict())

model2.load_state_dict(best_weights)

# Evaluate
lab_acc2 = evaluate_accuracy(model2, val_loader, device)
field_acc2 = evaluate_accuracy(model2, field_loader, device)
gap2 = lab_acc2 - field_acc2

print("\n" + "="*60)
print("📊 RESULTS: Passive Augmentation")
print("="*60)
print(f"🏠 Lab Accuracy:   {lab_acc2:.2f}%")
print(f"🌾 Field Accuracy: {field_acc2:.2f}%")
print(f"📉 GAP: {gap2:.2f}%")
print(f"\n📈 Improvement over baseline: {field_acc2 - exp01_results['field']:+.2f}%")
print("="*60)

exp02_results = {'lab': lab_acc2, 'field': field_acc2, 'gap': gap2}

---

## 4️⃣ Experiment 03: CutMix Augmentation

**Goal**: Test CutMix - cutting patches from one image and pasting onto another.

**Why it helps**: Forces the model to learn from multiple regions, improving robustness.

In [ ]:
# ============================================================
# EXPERIMENT 03: CutMix
# ============================================================

print("="*60)
print("🧪 EXPERIMENT 03: CutMix Augmentation")
print("="*60)

CUTMIX_PROB = 0.5  # Probability of applying CutMix
CUTMIX_BETA = 1.0  # Beta distribution parameter
CUTMIX_EPOCHS = 10  # CutMix needs more epochs

def rand_bbox(size, lam):
    """Generate random bounding box for CutMix."""
    W, H = size[2], size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w, cut_h = int(W * cut_rat), int(H * cut_rat)
    cx, cy = np.random.randint(W), np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

# Use standard transforms for CutMix
train_ds3 = FilteredImageFolder(str(lab_path), transforms_dict['train'], class_filter)
if not lab_val_path:
    train_size = int(0.8 * len(train_ds3))
    train_ds3, _ = random_split(train_ds3, [train_size, len(train_ds3) - train_size])

train_loader3 = DataLoader(train_ds3, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# Train with CutMix
print("\n🏋️ Training with CutMix...")
model3 = create_model(num_classes).to(device)
optimizer3 = optim.Adam(model3.parameters(), lr=LEARNING_RATE)
criterion3 = nn.CrossEntropyLoss()

best_acc3 = 0
best_weights3 = None

for epoch in range(CUTMIX_EPOCHS):
    model3.train()
    correct = total = 0
    
    for inputs, labels in train_loader3:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer3.zero_grad()
        
        # Apply CutMix with probability
        if np.random.rand() < CUTMIX_PROB:
            lam = np.random.beta(CUTMIX_BETA, CUTMIX_BETA)
            rand_index = torch.randperm(inputs.size(0)).to(device)
            labels_a, labels_b = labels, labels[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            outputs = model3(inputs)
            loss = criterion3(outputs, labels_a) * lam + criterion3(outputs, labels_b) * (1. - lam)
        else:
            outputs = model3(inputs)
            loss = criterion3(outputs, labels)
        
        loss.backward()
        optimizer3.step()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    
    train_acc = correct / total
    val_acc = evaluate_accuracy(model3, loaders['val'], device) / 100
    print(f"Epoch {epoch+1}/{CUTMIX_EPOCHS} | Train: {train_acc:.4f} (mixed) | Val: {val_acc:.4f}")
    
    if val_acc > best_acc3:
        best_acc3 = val_acc
        best_weights3 = copy.deepcopy(model3.state_dict())

model3.load_state_dict(best_weights3)

# Evaluate
lab_acc3 = evaluate_accuracy(model3, loaders['val'], device)
field_acc3 = evaluate_accuracy(model3, loaders['test'], device)
gap3 = lab_acc3 - field_acc3

print("\n" + "="*60)
print("📊 RESULTS: CutMix")
print("="*60)
print(f"🏠 Lab Accuracy:   {lab_acc3:.2f}%")
print(f"🌾 Field Accuracy: {field_acc3:.2f}%")
print(f"📉 GAP: {gap3:.2f}%")
print(f"\n📈 Improvement over baseline: {field_acc3 - exp01_results['field']:+.2f}%")
print("="*60)

exp03_results = {'lab': lab_acc3, 'field': field_acc3, 'gap': gap3}

---

## 5️⃣ Experiment 04: Active Learning Comparison

**Goal**: Compare Random vs Entropy-based sample selection for active learning.

**Setup**:
- Start with baseline model
- Iteratively select field samples to "label"
- Fine-tune and evaluate

**Key Observation**: Entropy shows an early "dip" because it picks hard samples first.

In [ ]:
# ============================================================
# EXPERIMENT 04: Active Learning
# ============================================================

print("="*60)
print("🧪 EXPERIMENT 04: Active Learning Comparison")
print("="*60)

import torch.nn.functional as F
from torch.utils.data import Subset
from common import load_model, MODELS_DIR

FINE_TUNE_LR = 0.0001
EPOCHS_PER_ROUND = 5

def compute_entropy(model, loader, device):
    """Compute uncertainty scores."""
    model.eval()
    entropies = []
    with torch.no_grad():
        for inputs, _ in loader:
            outputs = model(inputs.to(device))
            probs = F.softmax(outputs, dim=1)
            entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=1)
            entropies.extend(entropy.cpu().numpy())
    return np.array(entropies)

def fine_tune(model, loader, epochs, device):
    """Fine-tune on labeled samples."""
    model.train()
    opt = optim.Adam(model.parameters(), lr=FINE_TUNE_LR)
    crit = nn.CrossEntropyLoss()
    for _ in range(epochs):
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            opt.zero_grad()
            loss = crit(model(inputs), labels)
            loss.backward()
            opt.step()
    return model

def run_al_simulation(strategy, pool_ds, test_ds):
    """Run active learning simulation."""
    print(f"\n▶ Strategy: {strategy.upper()}")
    
    # Load fresh baseline
    model = load_model(MODELS_DIR / 'baseline_model.pth', num_classes, device)
    
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)
    pool_indices = list(range(len(pool_ds)))
    labeled_indices = []
    results = []
    
    # Initial
    acc = evaluate_accuracy(model, test_loader, device)
    print(f"  0 labels: {acc:.2f}%")
    results.append(acc)
    
    cumulative = 0
    for round_num in range(NUM_ROUNDS):
        cumulative += BUDGET_PER_ROUND
        
        # Select samples
        if strategy == 'random':
            np.random.shuffle(pool_indices)
            selected = pool_indices[:BUDGET_PER_ROUND]
            pool_indices = pool_indices[BUDGET_PER_ROUND:]
        else:  # entropy
            pool_subset = Subset(pool_ds, pool_indices)
            pool_loader = DataLoader(pool_subset, batch_size=32, shuffle=False)
            uncertainties = compute_entropy(model, pool_loader, device)
            sorted_idx = np.argsort(uncertainties)[::-1]
            sorted_pool = [pool_indices[i] for i in sorted_idx]
            selected = sorted_pool[:BUDGET_PER_ROUND]
            pool_indices = sorted_pool[BUDGET_PER_ROUND:]
        
        labeled_indices.extend(selected)
        
        # Fine-tune
        train_subset = Subset(pool_ds, labeled_indices)
        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True)
        model = fine_tune(model, train_loader, EPOCHS_PER_ROUND, device)
        
        # Evaluate
        acc = evaluate_accuracy(model, test_loader, device)
        print(f"  {cumulative} labels: {acc:.2f}%")
        results.append(acc)
    
    return results

# Prepare field data
print("\n📂 Preparing field data...")
pool_ds = FilteredImageFolder(str(field_path), transforms_dict['train'], class_filter)
test_ds_al = FilteredImageFolder(str(field_path), transforms_dict['val'], class_filter)

# Split 80/20
np.random.seed(42)
indices = np.random.permutation(len(pool_ds))
split = int(0.8 * len(pool_ds))
pool_subset = Subset(pool_ds, indices[:split].tolist())
test_subset = Subset(test_ds_al, indices[split:].tolist())

print(f"✅ Pool: {len(pool_subset)} | Test: {len(test_subset)}")

# Run simulations
results_random = run_al_simulation('random', pool_subset, test_subset)
results_entropy = run_al_simulation('entropy', pool_subset, test_subset)

# Results table
print("\n" + "="*60)
print("📊 RESULTS: Active Learning Comparison")
print("="*60)
print(f"{'Labels':<10} | {'Random':<12} | {'Entropy':<12} | {'Diff'}")
print("-" * 50)

x_values = [0] + [BUDGET_PER_ROUND * (i+1) for i in range(NUM_ROUNDS)]
for i, x in enumerate(x_values):
    diff = results_entropy[i] - results_random[i]
    print(f"{x:<10} | {results_random[i]:>10.2f}% | {results_entropy[i]:>10.2f}% | {diff:+.2f}%")

print("="*60)

exp04_results = {'random': results_random, 'entropy': results_entropy}

---

## 6️⃣ Experiment 05: Hybrid Warm-Start (Our Proposed Method)

**Goal**: Combine Random + Entropy sampling to get the best of both.

**The Hybrid Strategy**:
- **Round 0**: 50% Random + 50% Entropy (warm start)
- **Later rounds**: Pure entropy sampling

**Why it works**: Avoids the early "dip" while still benefiting from uncertainty sampling.

In [ ]:
# ============================================================
# EXPERIMENT 05: Hybrid Warm-Start (PROPOSED METHOD)
# ============================================================

print("="*60)
print("🧪 EXPERIMENT 05: Hybrid Warm-Start (OUR METHOD)")
print("="*60)

def run_hybrid_simulation(pool_ds, test_ds):
    """Run hybrid active learning simulation."""
    print(f"\n▶ Strategy: HYBRID")
    
    model = load_model(MODELS_DIR / 'baseline_model.pth', num_classes, device)
    
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)
    pool_indices = list(range(len(pool_ds)))
    labeled_indices = []
    results = []
    
    # Initial
    acc = evaluate_accuracy(model, test_loader, device)
    print(f"  0 labels: {acc:.2f}%")
    results.append(acc)
    
    cumulative = 0
    for round_num in range(NUM_ROUNDS):
        cumulative += BUDGET_PER_ROUND
        
        if round_num == 0:
            # WARM START: 50% random + 50% entropy
            n_random = BUDGET_PER_ROUND // 2
            n_entropy = BUDGET_PER_ROUND - n_random
            print(f"  🔥 Warm start: {n_random} random + {n_entropy} entropy")
            
            # Random part
            np.random.shuffle(pool_indices)
            random_sel = pool_indices[:n_random]
            remaining = pool_indices[n_random:]
            
            # Entropy part
            pool_subset = Subset(pool_ds, remaining)
            pool_loader = DataLoader(pool_subset, batch_size=32, shuffle=False)
            uncertainties = compute_entropy(model, pool_loader, device)
            sorted_idx = np.argsort(uncertainties)[::-1]
            sorted_remaining = [remaining[i] for i in sorted_idx]
            entropy_sel = sorted_remaining[:n_entropy]
            pool_indices = sorted_remaining[n_entropy:]
            
            selected = random_sel + entropy_sel
        else:
            # Pure entropy for later rounds
            pool_subset = Subset(pool_ds, pool_indices)
            pool_loader = DataLoader(pool_subset, batch_size=32, shuffle=False)
            uncertainties = compute_entropy(model, pool_loader, device)
            sorted_idx = np.argsort(uncertainties)[::-1]
            sorted_pool = [pool_indices[i] for i in sorted_idx]
            selected = sorted_pool[:BUDGET_PER_ROUND]
            pool_indices = sorted_pool[BUDGET_PER_ROUND:]
        
        labeled_indices.extend(selected)
        
        # Fine-tune
        train_subset = Subset(pool_ds, labeled_indices)
        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True)
        model = fine_tune(model, train_loader, EPOCHS_PER_ROUND, device)
        
        # Evaluate
        acc = evaluate_accuracy(model, test_loader, device)
        print(f"  {cumulative} labels: {acc:.2f}%")
        results.append(acc)
    
    return results

# Run hybrid
results_hybrid = run_hybrid_simulation(pool_subset, test_subset)

# Final comparison
print("\n" + "="*60)
print("📊 FINAL RESULTS: All Strategies")
print("="*60)
print(f"{'Labels':<10} | {'Random':<12} | {'Entropy':<12} | {'Hybrid':<12}")
print("-" * 55)

for i, x in enumerate(x_values):
    r, e, h = results_random[i], results_entropy[i], results_hybrid[i]
    best = max(r, e, h)
    row = f"{x:<10} | {r:>10.2f}% | {e:>10.2f}% | {h:>10.2f}%"
    if h == best:
        row += " ⭐"
    print(row)

print("="*60)

# Winner
final_random = results_random[-1]
final_entropy = results_entropy[-1]
final_hybrid = results_hybrid[-1]

print(f"\n🏆 FINAL ACCURACY WITH {BUDGET_PER_ROUND * NUM_ROUNDS} LABELS:")
print(f"   Random:  {final_random:.2f}%")
print(f"   Entropy: {final_entropy:.2f}%")
print(f"   Hybrid:  {final_hybrid:.2f}% {'⭐ BEST' if final_hybrid >= max(final_random, final_entropy) else ''}")

exp05_results = {'random': results_random, 'entropy': results_entropy, 'hybrid': results_hybrid}

---

## 📊 Summary: All Experiments Comparison

In [ ]:
# ============================================================
# SUMMARY OF ALL EXPERIMENTS
# ============================================================

print("\n" + "="*70)
print("📊 COMPLETE EXPERIMENT SUMMARY")
print("="*70)

print("\n🔬 PASSIVE METHODS (No field data used):")
print("-" * 50)
print(f"{'Method':<25} | {'Lab Acc':<10} | {'Field Acc':<10} | {'Gap'}")
print("-" * 50)
print(f"{'Baseline':<25} | {exp01_results['lab']:>8.2f}% | {exp01_results['field']:>8.2f}% | {exp01_results['gap']:.2f}%")
print(f"{'+ Passive Augmentation':<25} | {exp02_results['lab']:>8.2f}% | {exp02_results['field']:>8.2f}% | {exp02_results['gap']:.2f}%")
print(f"{'+ CutMix':<25} | {exp03_results['lab']:>8.2f}% | {exp03_results['field']:>8.2f}% | {exp03_results['gap']:.2f}%")

print("\n🎯 ACTIVE METHODS (Using field data budget):")
print("-" * 50)
total_budget = BUDGET_PER_ROUND * NUM_ROUNDS
print(f"With {total_budget} labeled field images:")
print(f"  Random sampling:  {exp05_results['random'][-1]:.2f}%")
print(f"  Entropy sampling: {exp05_results['entropy'][-1]:.2f}%")
print(f"  Hybrid (Ours):    {exp05_results['hybrid'][-1]:.2f}%")

print("\n" + "="*70)
print("🏆 KEY FINDINGS:")
print("="*70)
print(f"1. Baseline gap: {exp01_results['gap']:.1f}% accuracy drop from Lab to Field")
print(f"2. Passive augmentation helps but doesn't solve the problem")
print(f"3. Active learning with {total_budget} labels significantly improves field accuracy")
print(f"4. Hybrid warm-start achieves the best results")
print("="*70)

---

## 📈 Visualization

In [ ]:
# ============================================================
# PLOT RESULTS
# ============================================================

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Passive methods comparison
ax1 = axes[0]
methods = ['Baseline', 'Passive Aug', 'CutMix']
lab_accs = [exp01_results['lab'], exp02_results['lab'], exp03_results['lab']]
field_accs = [exp01_results['field'], exp02_results['field'], exp03_results['field']]

x = np.arange(len(methods))
width = 0.35

bars1 = ax1.bar(x - width/2, lab_accs, width, label='Lab', color='#2ecc71')
bars2 = ax1.bar(x + width/2, field_accs, width, label='Field', color='#e74c3c')

ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Passive Methods Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(methods)
ax1.legend()
ax1.set_ylim(0, 100)

# Add value labels
for bar in bars1 + bars2:
    height = bar.get_height()
    ax1.annotate(f'{height:.1f}%', xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

# Plot 2: Active learning curves
ax2 = axes[1]
ax2.plot(x_values, exp05_results['random'], 'o-', label='Random', color='gray', linewidth=2)
ax2.plot(x_values, exp05_results['entropy'], 's-', label='Entropy', color='#f39c12', linewidth=2)
ax2.plot(x_values, exp05_results['hybrid'], '^-', label='Hybrid (Ours)', color='#e74c3c', linewidth=2, markersize=8)

ax2.set_xlabel('Number of Labeled Field Images')
ax2.set_ylabel('Field Test Accuracy (%)')
ax2.set_title('Active Learning Strategies Comparison')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(project_root / 'results' / 'figures' / 'notebook_results.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Figure saved to results/figures/notebook_results.png")

---

## ✅ Done!

You have successfully run all experiments. The key takeaways are:

1. **The Problem**: Models trained on lab data lose ~60-70% accuracy on field data
2. **Passive solutions** (augmentation, CutMix) help slightly but don't solve the problem
3. **Active learning** with a small labeling budget significantly improves results
4. **Our Hybrid method** achieves the best performance by avoiding the entropy "dip"

---

**Next Steps**:
- Try different class filters (e.g., `CLASS_NAME = 'Apple'`)
- Adjust the labeling budget to see its effect
- Run with more epochs for better results